In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import torch
from actuallysparse import converter
from warnings import filterwarnings
from torch.autograd.profiler import profile

filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
model_loader_very_pruned = lambda: torch.load("../.weights/full/very_pruned", map_location=torch.device('cpu')).eval()
model_loader_pruned = lambda: torch.load("../.weights/full/pruned", map_location=torch.device('cpu')).eval()
model_loader_pretrained = lambda: torch.load("../.weights/full/pretrained", map_location=torch.device('cpu')).eval()

In [8]:
dummy_input = torch.ones(1,3,244,244)
dense = model_loader_pruned().to(device).eval()
coo = converter.convert_model(model_loader_pruned(), torch.nn.Linear, "coo").to(device)
coo.train(False)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [9]:
with profile(profile_memory=True, record_shapes=True) as prof:
        dense(dummy_input)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.04%      52.000us        81.50%     104.810ms      13.101ms      33.46 Mb           0 b             8  
                aten::convolution         0.15%     194.000us        81.46%     104.758ms      13.095ms      33.46 Mb           0 b             8  
               aten::_convolution         0.07%      95.000us        81.31%     104.564ms      13.070ms      33.46 Mb           0 b             8  
         aten::mkldnn_convolution        81.05%     104.238ms        81.23%     104.469ms      13.059ms      33.

In [10]:
with profile(profile_memory=True, record_shapes=True) as prof:
        coo(dummy_input)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

--------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
--------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                      aten::conv2d         0.03%      62.000us        56.60%     109.471ms      13.684ms      33.46 Mb           0 b             8  
                                 aten::convolution         0.10%     201.000us        56.57%     109.409ms      13.676ms      33.46 Mb           0 b             8  
                                aten::_convolution         0.06%     109.000us        56.46%     109.208ms      13.651ms      33.46 Mb           0 b             8  
          

In [11]:
dense = model_loader_very_pruned().to(device).eval()
coo = converter.convert_model(model_loader_very_pruned(), torch.nn.Linear, "coo").to(device)
coo.train(False)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [12]:
coo.train(False)
with profile(profile_memory=True, record_shapes=True) as prof:
    dense(dummy_input)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.04%      55.000us        81.71%     107.350ms      13.419ms      33.46 Mb           0 b             8  
                aten::convolution         0.16%     209.000us        81.67%     107.295ms      13.412ms      33.46 Mb           0 b             8  
               aten::_convolution         0.08%     103.000us        81.51%     107.086ms      13.386ms      33.46 Mb           0 b             8  
         aten::mkldnn_convolution        81.26%     106.753ms        81.44%     106.983ms      13.373ms      33.

In [13]:
with profile(profile_memory=True, record_shapes=True) as prof:
        coo(dummy_input)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

--------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
--------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                      aten::conv2d         0.04%      54.000us        82.45%     108.139ms      13.517ms      33.46 Mb           0 b             8  
                                 aten::convolution         0.15%     192.000us        82.41%     108.085ms      13.511ms      33.46 Mb           0 b             8  
                                aten::_convolution         0.08%     110.000us        82.26%     107.893ms      13.487ms      33.46 Mb           0 b             8  
          